# Notebook to generate VHDL for SAD

In [1]:
import numpy as np 
import math

In [82]:
################################################
################## Parametros ##################
block_size = 4
bit_width = 8
proc_parallel = True        # True: processamento paralelo de bloco todo, False: processamento por linha
files = ['adder.vhd']       # Lista de componentes para add

if proc_parallel:
    file_name = ('sad_' + str(int(math.sqrt(block_size))) + 'x' + str(int(math.sqrt(block_size))) + '_parallel')
else:
    file_name = ('sad_' + str(int(math.sqrt(block_size))) + 'x' + str(int(math.sqrt(block_size))) + '_line')

pack_name = 'type_pack'
rewrite_package = False
################################################

In [83]:
################################################# Port ################################################
port = ('''pixels_O: in input_line;
    pixels_R: in input_line;
    sad_out: out std_logic_vector(''' + str(math.ceil(np.log2(block_size))+bit_width-1) + ''' downto 0)''')

################################################# Components ################################################
components = []
line_qnt = 0

for comp in files:
    with open(comp, "r") as file:
        lines = file.readlines()

    line_split = []
    for i in range(len(lines)):                # Separa os componentes de cada linha lida
        line_split.append(lines[i].split())

    end_line = 0
    for i in range(0,len(lines),1):                 # Itera em todas linhas        
        if line_split[i] != []:                     # Verifica se não é uma linha vazia
            if line_split[i][0] == 'entity':        # Vê se é entity                
                begin_line = i
            elif line_split[i][0] == 'end':         # Vê se é o final de entity
                if end_line == 0:
                    end_line = i

    for l in range(begin_line,end_line+1,1):        # Pega todas as linhas da entity
        components.append(lines[l])

    line1_split = components[line_qnt].split()      # Alterando a palavra 'entity' 
    line1_split[0] = 'component'                    # para 'component'
    line1_split.append('\n')
    components[line_qnt] = ' '.join(line1_split)

    linelast_split = components[-1].split()         # Alterando a última palavra 
    linelast_split[-1] = 'component;'               # para 'component'
    components[-1] = ' '.join(linelast_split)

    components.append('\n\n')
    line_qnt = len(components)

components_concat = ' '.join(components)         #  Junta todos componentes

################################################# Signals ################################################
if proc_parallel:                           # Ex: bloco 4x4 processa 16 em paralelo
    limit = block_size
else:
    limit = int(math.sqrt(block_size))      # Ex: bloco 4x4 processa 4 vezes cada linha de 4

sub = []    # Sinais da 1a subtracao (estrutura basica)
for i in range(0,limit):
    sub.append('signal sub_' + str(i) + ': std_logic_vector(' + str(bit_width) + ' downto 0);')
sub.append('\n')
sub_concat = '\n'.join(sub)

abs = []    # Sinais do calculo do absoluto (estrutura basica)
for i in range(0,limit):
    abs.append('signal abs_' + str(i) + ': std_logic_vector(' + str(bit_width-1) + ' downto 0);')
abs.append('\n')
abs_concat = '\n'.join(abs)

sum0 = []   # Sinais da 1a soma (estrutura basica)
for i in range(0,limit,2):
    sum0.append('signal sum_' + str(i) + str(i+1) + ': std_logic_vector(' + str(bit_width) + ' downto 0);')
sum0.append('\n')
sum0_concat = '\n'.join(sum0)

after_sums = [] # Sinais dos outros niveis de soma abaixo 
for k in range(1,math.ceil(np.log2(limit))):    # Qtd de niveis de soma
    qtd_sum = int(limit/(2**(k+1)))             # Qtd de somados por nivel
    for j in range(0,qtd_sum):
        after_sums.append('signal sum_after_'+str(k)+str(j)+': std_logic_vector('+str(bit_width+k)+' downto 0);')
after_sums.append('\n')
after_sums_concat = '\n'.join(after_sums)

signals = []
signals.append(sub_concat + abs_concat + sum0_concat + after_sums_concat)
signals_concat = ''.join(signals)




In [84]:
# Texto do VHDL
vhdl = ('''library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_unsigned.all;
use ieee.numeric_std.all;
use work.''' + pack_name + '''.all;

entity ''' + file_name + ''' is 
port(
    ''' + port + '''
);
end ''' + file_name + ''';

architecture arch_sad of ''' + file_name + ''' is 

-- Components 
''' + components_concat + '''
-- Signals
''' + signals_concat + '''

begin'''
)

print(vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_unsigned.all;
use ieee.numeric_std.all;
use work.type_pack.all;

entity sad_2x2_parallel is 
port(
    pixels_O: in input_line;
    pixels_R: in input_line;
    sad_out: out std_logic_vector(9 downto 0)
);
end sad_2x2_parallel;

architecture arch_sad of sad_2x2_parallel is 

-- Components 
component adder is 
 generic (N: integer);
 port(
 		A, B: in std_logic_vector(N-1 downto 0);
 		S: out std_logic_vector(N downto 0)
 );
 end component; 


-- Signals
signal sub_0: std_logic_vector(8 downto 0);
signal sub_1: std_logic_vector(8 downto 0);
signal sub_2: std_logic_vector(8 downto 0);
signal sub_3: std_logic_vector(8 downto 0);

signal abs_0: std_logic_vector(7 downto 0);
signal abs_1: std_logic_vector(7 downto 0);
signal abs_2: std_logic_vector(7 downto 0);
signal abs_3: std_logic_vector(7 downto 0);

signal sum_01: std_logic_vector(8 downto 0);
signal sum_23: std_logic_vector(8 downto 0);

signal sum_after_10: std_logic_vect

abs_0 <= std_logic_vector(abs(signed((sub_0)));
abs_1 <= std_logic_vector(abs(signed((sub_1)));
abs_2 <= std_logic_vector(abs(signed((sub_2)));
abs_3 <= std_logic_vector(abs(signed((sub_3)));

## Geração do Package

In [ ]:
# Criando package de tipo
pack = ('''library ieee;
use ieee.std_logic_1164.all;
use ieee.std_logic_unsigned.all;
use ieee.numeric_std.all;

package ''' + pack_name + ''' is
    
    constant bit_width: natural := ''' + str(bit_width) + '''; 
    type input_line is array (0 to ''' + str(block_size-1) + ") of std_logic_vector(" + str(bit_width-1) + ''' downto 0);

end ''' + pack_name + ';'
)

if rewrite_package:
    pack_file = open((pack_name + ".vhd"), "w")
    pack_file.write(pack)
    pack_file.close()